# UK Influenza Hospital Admissions Explorer

Interactive exploration of weekly influenza hospital admission rates in England using UKHSA SARI-Watch sentinel data.

**Data source**: UK Health Security Agency (UKHSA) Public Dashboard API  
**Metric**: Influenza hospital admission rate per 100,000 population (weekly)  
**Current season**: 2025–26 (as of December 2025 – rising A(H3N2) activity)

This notebook loads the data generated by `fetch_and_analyze.py` and allows interactive visualisation and comparison.

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the CSV generated by the main script
df = pd.read_csv('flu_data.csv', parse_dates=['date'])

# Quick check
print(df.shape)
df.head()

## Data Overview

- **date**: End of specimen week
- **value**: Admission rate per 100,000 trust catchment population
- **rolling_7_week_avg**: Smoothed trend
- **season_label**: Flu season ( 2025-26 = Oct 2025 – Sep 2026)

df.describe()

plt.figure(figsize=(14, 7))
sns.set_style("whitegrid")
sns.lineplot(data=df, x='date', y='value', label='Weekly Rate')
sns.lineplot(data=df, x='date', y='rolling_7_week_avg', label='7-Week Rolling Average', color='orange', linewidth=2)

# Highlight current season
current_season = df['season_label'].max()
sns.lineplot(data=df[df['season_label'] == current_season], x='date', y='value', 
             color='red', linewidth=3, label=f'{current_season} (current)')

plt.title('England Influenza Hospital Admission Rates Over Time')
plt.xlabel('Date')
plt.ylabel('Rate per 100,000')
plt.legend()
plt.show()

## Seasonal Comparison

Flu seasons aligned by ISO week number (week 40 ≈ October start).  
Allows direct comparison of epidemic curves across years.

plt.figure(figsize=(16, 8))
sns.set_style("whitegrid")

seasons = sorted(df['season_label'].unique())
palette = sns.color_palette("husl", len(seasons))

for i, season in enumerate(seasons):
    season_data = df[df['season_label'] == season]
    color = 'red' if season == seasons[-1] else palette[i]
    linewidth = 4 if season == seasons[-1] else 2
    label = f'{season} (current)' if season == seasons[-1] else season
    sns.lineplot(data=season_data, x='week', y='value', label=label, color=color, linewidth=linewidth)

# Current season rolling average
current = df[df['season_label'] == seasons[-1]]
sns.lineplot(data=current, x='week', y='rolling_7_week_avg', color='darkred', 
             linestyle='--', linewidth=2, label=f'{seasons[-1]} 7-week avg')

plt.title('Seasonal Comparison: Influenza Hospital Admissions by Week')
plt.xlabel('ISO Week Number')
plt.ylabel('Rate per 100,000')
plt.xlim(35, 52)
plt.legend(title='Season')
plt.show()

latest = df.iloc[-1]
print(f"Latest data: {latest['date'].date()} (week {int(latest['week'])})")
print(f"Admission rate: {latest['value']:.2f} per 100,000")
print(f"Compared to same week last season: {df[(df['week'] == latest['week']) & (df['season_label'] == '2024-25')]['value'].values[0]:.2f}" 
      if not df[(df['week'] == latest['week']) & (df['season_label'] == '2024-25')].empty else "No prior data for this week")

## Notes & Limitations

- Data from **SARI-Watch sentinel surveillance** (subset of trusts, voluntary reporting).
- Rates are per 100,000 trust catchment population.
- Updated weekly (usually Thursdays/Fridays).
- Source: UKHSA National Flu and COVID-19 Surveillance Report (latest: 18 December 2025, week 51).

**License**: Code – MIT | Data – Open Government Licence v3.0